In [3]:
import ee

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Qdh0EvkeYFR722SlFAjHqoWvzfyM98dL7nhl8jacXpo&tc=1rh4MaEZ8cUofP6NVTAU3MUWs7YJzWM5oM0tWjjRJjI&cc=vp7cZs6pZ_reA-cBD0XwAPQQmENBRUqVbObuL9Y6n74

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh40U7bUu52w7TQLG42D3vhHaLKn3QQghrh-0RO32GydjSmSHH8XF7A

Successfully saved authorization token.


In [5]:
def col2monthly(col):
    
    def monthly(m):
        return (col.filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean()
                .set('month', m)
                .set('system:index', ee.Number(m).format('%02d'))
                )

    months = ee.List.sequence(1, 12)
    col_m = ee.ImageCollection.fromImages(months.map(monthly).flatten())
    
    return col_m

def grid2scale(img):
    return img.reproject(crs='EPSG:4326', scale=img.projection().nominalScale())#.clip(refpoly)


In [6]:
# Precipitation
# gives mm/month
y_start = 2001
y_end = 2020

p = col2monthly(ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06")
                .select(['precipitation'], ['P'])
                .filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                ) # monthly precipitation

p_avg = p.mean().multiply(24*30).rename('p_avg')
p_min = p.min().multiply(24*30).rename('p_min')
p_max = p.max().multiply(24*30).rename('p_max')

In [7]:
# Temperature
# gives celsius
y_start = 2001
y_end = 2020

t = col2monthly(ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY")
                .select(['temperature_2m'], ['T'])
                .filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                )

t_avg = t.mean().rename('t_avg').subtract(273.15)
t_min = t.min().rename('t_min').subtract(273.15)
t_max = t.max().rename('t_max').subtract(273.15)

In [8]:
# ET
# Gives mm
y_start = 2001
y_end = 2020

et = col2monthly(ee.ImageCollection("MODIS/NTSG/MOD16A2/105")
                .select('ET')
                .filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                )

et_avg = et.mean().rename('et_avg').divide(80)
et_min = et.min().rename('et_min').divide(80)
et_max = et.max().rename('et_max').divide(80)

In [9]:
# Elevation and Slope
import math

dem = ee.Image("MERIT/DEM/v1_0_3").select('dem')
slp = ee.Terrain.slope(dem).divide(180).multiply(ee.Number(math.pi)).tan().multiply(100).rename('slp')

In [10]:
# Soil data

soil_texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02").select('b0').rename('soiltext')

soil_clay_content = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').rename('soilclay')
soil_sand_content = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').rename('soilsand')
soil_organic_content = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').rename('soilorga')

soil_density = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").select('b0').rename('soildens')

In [11]:
# Land cover
# MODIS #####
# 1 Evergreen Needleleaf Forests
# 2 Evergreen Broadleaf Forests
# 3 Deciduous Needleleaf Forests
# 4 Deciduous Broadleaf Forests
# 5 Mixed Forests
# 6 Closed Shrublands
# 7 Open Shrublands
# 8 Woody Savannas
# 9 Savannas
# 10 Grasslands
# 11 Permanent Wetlands
# 12 Croplands
# 13 Urban and Built-up Lands
# 14 Cropland/Natural Vegetation Mosaics
# 15 Permanent Snow and Ice
# 16 Barren
# 17 Water Bodies

year = 2010
imgcol = ee.ImageCollection("MODIS/006/MCD12Q1").select('LC_Type1')
img1 = imgcol.filter(ee.Filter.calendarRange(year, year, 'year')).first()
img = img1.reproject('EPSG:4326', scale=img1.projection().nominalScale())

old_classes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
new_classes = [1, 1, 1, 1, 1, 2, 2, 2, 3,  3,  3,  3,  4,  3,  4,  4,  5]
lc = img.remap(old_classes, new_classes).rename('LC')

In [14]:
def imgExtract(img, img_name, scale, refpoly, folder):
    img = img.clip(refpoly)
    
    task_config = {
            'image': img,
            'description': img_name,
            'folder': folder,
            'scale': scale,
            'region': refpoly,
            'maxPixels': 1e12
            }
    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()

    return

def fcExtract(fc, description, folder):

    task_config = {
      'collection': fc,
      'folder': folder,
      'description': description,
      'fileFormat': 'CSV',
      # 'selectors': selectors
          }
    task = ee.batch.Export.table.toDrive(**task_config)
    task.start()

    return

def img2scale(img, scale):
    return img.reproject('EPSG:4326', scale=scale)

def remove_geom(feat):
    return feat.setGeometry(None)

# Region of Interest
Fc = ee.FeatureCollection("FAO/GAUL/2015/level0");
folder = 'GEE_intro'

# Agg climate features (P and T and E)    
img = p.toBands().multiply(24*30)
Fc = img2scale(img, 1000).reduceRegions(Fc, ee.Reducer.mean())

img = t.toBands().subtract(273.15)
Fc = img2scale(img, 1000).reduceRegions(Fc, ee.Reducer.mean())

img = et.toBands()
Fc = img2scale(img, 1000).reduceRegions(Fc, ee.Reducer.mean())

# Agg topography features
img = dem.rename('elv_avg').addBands(slp.rename('slp_avg'))
Fc = img2scale(img, 90).reduceRegions(Fc, ee.Reducer.mean())

img = dem.rename('elv_std').addBands(slp.rename('slp_std'))
Fc = img2scale(img, 90).reduceRegions(Fc, ee.Reducer.stdDev())

# Agg soil and land cover features
img = soil_density.addBands(soil_clay_content).addBands(soil_sand_content).addBands(soil_organic_content)
Fc = img2scale(img, 250).reduceRegions(Fc, ee.Reducer.mean())

img = lc.addBands(soil_texture)
Fc = img2scale(img, 250).reduceRegions(Fc, ee.Reducer.frequencyHistogram())

# Code to extract variables to basins
# If memory limit exceeded, do it in chunks accordingly (trial and error)
fcExtract(Fc.map(remove_geom), 'countries_features', folder)